In [248]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import ipaddress

In [249]:
def to_int(x):
    return int(x, 16)

In [250]:
path = "./datasets/iot-network-intrusion-dataset_190919/Packets_integrated/"

In [251]:
names_list = [f for f in os.listdir(path) if f.endswith('.txt')]

In [252]:
names_list

['benign-dec.txt',
 'dos-synflooding.txt',
 'mirai-ackflooding.txt',
 'mirai-hostbruteforce-1,3,5.txt',
 'mirai-hostbruteforce-2,4.txt',
 'mirai-httpflooding.txt',
 'mirai-udpflooding.txt',
 'mitm.txt',
 'scan-hostport_1.txt',
 'scan-hostport_2.txt',
 'scan-hostport_3.txt',
 'scan-hostport_4.txt',
 'scan-hostport_5.txt',
 'scan-hostport_6.txt',
 'scan-portos.txt']

# 파일 변환 코드

In [253]:
# for txt in names_list:
#     packet = pd.read_csv(os.path.join(path, txt), sep="\n", header = None)
#     packet[packet[0].str.contains("Frame \(")] = "\f"
#     packet[0] = packet[0].str[:53]
#     packet = packet[0].str.split("  ", expand=True)
#     packet = packet[[0, 1]]
#     packet = pd.DataFrame(packet.to_string(header=None, index=False).split("\f"))
#     print("Almost Done!")
#     packet[0]=packet[0].str.lstrip()
#     packet[0] = packet[0].str.replace("\n", " ")
#     for _ in range(10):
#         packet[0] = packet[0].str.replace("  ", " ")
#     packet[0] = packet[0].str[:222]
#     packet = packet[0].str.split(" ", expand=True)
#     packet=packet.fillna("00")
#     packet = packet.drop(columns=[0, 17, 34, 51, 68, 71])
#     packet=packet.replace("","00")
#     packet.to_csv(os.path.join(path, txt).replace(".txt","_hex")+".csv")
#     print(txt,"done")
# #     packet = packet.applymap(to_int)

# Reading files

In [254]:
hex_list = [f for f in os.listdir(path) if f.endswith('hex.csv')]

In [255]:
hex_list

['benign-dec_hex.csv',
 'dos-synflooding_hex.csv',
 'mirai-ackflooding_hex.csv',
 'mirai-hostbruteforce-1,3,5_hex.csv',
 'mirai-hostbruteforce-2,4_hex.csv',
 'mirai-httpflooding_hex.csv',
 'mirai-udpflooding_hex.csv',
 'mitm_hex.csv',
 'scan-hostport_1_hex.csv',
 'scan-hostport_2_hex.csv',
 'scan-hostport_3_hex.csv',
 'scan-hostport_4_hex.csv',
 'scan-hostport_5_hex.csv',
 'scan-hostport_6_hex.csv',
 'scan-portos_hex.csv']

In [256]:
dict_names=hex_list.copy()
csv_list=hex_list.copy()
for i in range(len(hex_list)):
    dict_names[i] = hex_list[i].replace("_hex.csv","")
    csv_list[i] = hex_list[i].replace("_hex","")

In [257]:
dict_names

['benign-dec',
 'dos-synflooding',
 'mirai-ackflooding',
 'mirai-hostbruteforce-1,3,5',
 'mirai-hostbruteforce-2,4',
 'mirai-httpflooding',
 'mirai-udpflooding',
 'mitm',
 'scan-hostport_1',
 'scan-hostport_2',
 'scan-hostport_3',
 'scan-hostport_4',
 'scan-hostport_5',
 'scan-hostport_6',
 'scan-portos']

In [258]:
csv_list

['benign-dec.csv',
 'dos-synflooding.csv',
 'mirai-ackflooding.csv',
 'mirai-hostbruteforce-1,3,5.csv',
 'mirai-hostbruteforce-2,4.csv',
 'mirai-httpflooding.csv',
 'mirai-udpflooding.csv',
 'mitm.csv',
 'scan-hostport_1.csv',
 'scan-hostport_2.csv',
 'scan-hostport_3.csv',
 'scan-hostport_4.csv',
 'scan-hostport_5.csv',
 'scan-hostport_6.csv',
 'scan-portos.csv']

In [259]:
X_dict = {}
for i, packets in enumerate(hex_list):
    X_dict[dict_names[i]] = pd.read_csv(os.path.join(path, packets),index_col=0, dtype=str)
    X_dict[dict_names[i]] = X_dict[dict_names[i]].join(pd.read_csv(os.path.join(path,csv_list[i])),how="outer")

In [260]:
X_dict['mirai-hostbruteforce-1,3,5']

,1,2,3,4,5,6,7,8,9,10,...,67,69,70,No.,Time,Source,Destination,Protocol,Length,Info
0,04,32,f4,45,17,b3,88,36,6c,d7,...,ff,fb,e1,1,0.000000e+00,104.74.213.186,192.168.0.24,TCP,826,[TCP segment of a reassembled PDU]
1,04,32,f4,45,17,b3,88,36,6c,d7,...,ff,fb,e1,2,2.200000e-04,104.74.213.186,192.168.0.24,TCP,1514,[TCP segment of a reassembled PDU]
2,04,32,f4,45,17,b3,88,36,6c,d7,...,ff,fb,e1,3,5.600000e-04,104.74.213.186,192.168.0.24,TCP,1514,[TCP segment of a reassembled PDU]
3,04,32,f4,45,17,b3,88,36,6c,d7,...,ff,fb,e1,4,6.370000e-04,104.74.213.186,192.168.0.24,TCP,1514,[TCP segment of a reassembled PDU]
4,04,32,f4,45,17,b3,88,36,6c,d7,...,ff,fb,e1,5,1.378000e-03,104.74.213.186,192.168.0.24,TCP,1514,[TCP segment of a reassembled PDU]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273352,88,36,6c,d7,1c,56,04,32,f4,45,...,c9,de,65,273353,1.810277e+06,192.168.0.24,211.237.6.104,TCP,66,58485 > 443 [ACK] Seq=2510 Ack=376686 Win=52...
273353,04,32,f4,45,17,b3,88,36,6c,d7,...,00,00,00,273354,1.810277e+06,EFMNetwo_d7:1c:56,Partron_45:17:b3,ARP,42,Who has 192.168.0.24? Tell 192.168.0.1
273354,88,36,6c,d7,1c,56,04,32,f4,45,...,00,00,00,273355,1.810277e+06,Partron_45:17:b3,EFMNetwo_d7:1c:56,ARP,42,192.168.0.24 is at 04:32:f4:45:17:b3
273355,bc,1c,81,4b,ae,ba,88,36,6c,d7,...,00,00,00,273356,1.810277e+06,EFMNetwo_d7:1c:56,Sichuani_4b:ae:ba,ARP,42,Who has 192.168.0.13? Tell 192.168.0.1


In [261]:
y_dict = {}
for key, value in enumerate(X_dict):
    y_dict[value] = pd.read_csv(os.path.join(path, value)+"_attacked.csv", header=None)[0][1:].apply(int)

C:\Users\hjhhi\anaconda3\envs\python3.6.8\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,1,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [262]:
for i in dict_names:
    print(i, len(X_dict[i]),"    attacked", len(y_dict[i]))

benign-dec 137396     attacked 0
dos-synflooding 141709     attacked 64646
mirai-ackflooding 313462     attacked 75632
mirai-hostbruteforce-1,3,5 273357     attacked 1636
mirai-hostbruteforce-2,4 179998     attacked 961
mirai-httpflooding 248294     attacked 10464
mirai-udpflooding 1187114     attacked 949284
mitm 194184     attacked 101885
scan-hostport_1 29352     attacked 1490
scan-hostport_2 37106     attacked 2701
scan-hostport_3 13555     attacked 1730
scan-hostport_4 7801     attacked 1707
scan-hostport_5 4760     attacked 2211
scan-hostport_6 6828     attacked 2401
scan-portos 211078     attacked 12970


In [263]:
label_dict={}
for i in dict_names:
    label = np.zeros(len(X_dict[i]))
    label[y_dict[i]-1] = 1
    ###위에 -1 붙여야됨
    label_dict[i] = label

In [264]:
X_dict['benign-dec']

,1,2,3,4,5,6,7,8,9,10,...,67,69,70,No.,Time,Source,Destination,Protocol,Length,Info
0,48,4b,aa,2c,d8,f9,bc,1c,81,4b,...,d3,9b,8a,1,0.000000,192.168.0.13,192.168.0.16,TCP,1498,9020 > 49784 [ACK] Seq=5029255 Ack=1 Win=186...
1,48,4b,aa,2c,d8,f9,bc,1c,81,4b,...,d3,9b,8a,2,0.000144,192.168.0.13,192.168.0.16,TCP,140,"9020 > 49784 [PSH, ACK] Seq=5030643 Ack=1 Wi..."
2,48,4b,aa,2c,d8,f9,bc,1c,81,4b,...,d3,9b,8a,3,0.000227,192.168.0.13,192.168.0.16,TCP,1502,9020 > 49784 [ACK] Seq=5030673 Ack=1 Win=186...
3,48,4b,aa,2c,d8,f9,bc,1c,81,4b,...,d3,9b,8a,4,0.000765,192.168.0.13,192.168.0.16,TCP,1498,9020 > 49784 [ACK] Seq=5032061 Ack=1 Win=186...
4,48,4b,aa,2c,d8,f9,bc,1c,81,4b,...,d3,9b,8a,5,0.000840,192.168.0.13,192.168.0.16,TCP,1502,9020 > 49784 [ACK] Seq=5033449 Ack=1 Win=186...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137391,bc,1c,81,4b,ae,ba,48,4b,aa,2c,...,07,96,8a,137392,301.084925,192.168.0.16,192.168.0.13,TCP,114,49799 > 9020 [ACK] Seq=193 Ack=1018277 Win=5...
137392,48,4b,aa,2c,d8,f9,bc,1c,81,4b,...,d8,17,b5,137393,301.085935,192.168.0.13,192.168.0.16,TCP,434,"9020 > 49799 [PSH, ACK] Seq=1018277 Ack=193 ..."
137393,48,4b,aa,2c,d8,f9,bc,1c,81,4b,...,d8,17,b5,137394,301.086577,192.168.0.13,192.168.0.16,TCP,434,"[TCP Retransmission] 9020 > 49799 [PSH, ACK]..."
137394,bc,1c,81,4b,ae,ba,48,4b,aa,2c,...,07,96,8a,137395,301.088906,192.168.0.16,192.168.0.13,TCP,114,49799 > 9020 [ACK] Seq=193 Ack=1018597 Win=5...


In [265]:
X_reduced = {}
for i in X_dict.keys():
    X_dict[i]["SourcePort"] = (X_dict[i]["37"]+X_dict[i]["38"]).apply(to_int)
    X_dict[i]["DestinationPort"] = (X_dict[i]["39"]+X_dict[i]["40"]).apply(to_int)
    X_reduced[i] = pd.DataFrame()
    X_reduced[i]["SourcePort"] = X_dict[i]["SourcePort"]    
    X_reduced[i]["DestinationPort"] = X_dict[i]["DestinationPort"]
    X_reduced[i]["Source"] = X_dict[i]["Source"]        
    X_reduced[i]["Destination"] = X_dict[i]["Destination"]
    X_reduced[i]["Time"] = X_dict[i]["Time"]        

In [266]:
# print(X_reduced["Source"].isna().sum(),
# X_reduced[i]["SourcePort"].isna().sum(),
# X_reduced[i]["Destination"].isna().sum(),
# X_reduced[i]["DestinationPort"].isna().sum(),
# X_reduced[i]["Time"].isna().sum())

In [267]:
X_dict['mirai-udpflooding']["Protocol"].unique()

array(['TCP', 'SSLv2', 'TLSv1.2', 'DNS', 'TLSv1', 'UDP', 'ARP', 'IPv4',
       'SSDP', '0x81d9', 'SSL', 'HTTP', 'IGMPv2', '0x46d0', 'ICMP',
       '0xcd9c', 'MDNS', 'ICMPv6', 'TLSv1.3', 'GQUIC', '0x1700',
       'HTTP/XML', '0x50a4', '0x0873', '0x3a24', '0x8900', '0x1800',
       'SSLv3', 'UDT', 'IPv6', '0x0801', '0x0880', '0x0a00', '0x8820',
       '0x0840', '0x08a0', '0x2000', 'LLC', '0xd816', 'NTP', '0x0820',
       '0x38e9', '0x8604', '0xc834', '0x8800', '0x0804', '0x4b00',
       'Gryphon', '0xea84', '0x0860', '0xa280', '0xc891', '0x8810',
       '0x4200', '0x858a', '0x0870', '0x9003', '0xa300', '0x4843',
       '0x0811', '0x0802', '0x0883', '0xa491', '0xc2c6', '0x0808',
       '0x0868', '0x3754', '0x6164', 'X11', '0x4eaa', '0x1020', '0x8823',
       '0xc4ef', '0xf560', '0x1805', '0x4065', '0x0810', '0x2200',
       'WTLS+WTP+WSP', '0xe000', '0x0a10', '0xe784', '0x5825', '0x881f',
       '0x49bf', '0x4840', '0x3e27', '0x3d02', '0x0000', '0x0920',
       'BROWSER', 'NBNS', '0xc0e2'

In [268]:
X_reduced['benign-dec']

,SourcePort,DestinationPort,Source,Destination,Time
0,9020,49784,192.168.0.13,192.168.0.16,0.000000
1,9020,49784,192.168.0.13,192.168.0.16,0.000144
2,9020,49784,192.168.0.13,192.168.0.16,0.000227
3,9020,49784,192.168.0.13,192.168.0.16,0.000765
4,9020,49784,192.168.0.13,192.168.0.16,0.000840
...,...,...,...,...,...
137391,49799,9020,192.168.0.16,192.168.0.13,301.084925
137392,9020,49799,192.168.0.13,192.168.0.16,301.085935
137393,9020,49799,192.168.0.13,192.168.0.16,301.086577
137394,49799,9020,192.168.0.16,192.168.0.13,301.088906


In [269]:
flooding_list=['dos-synflooding','mirai-ackflooding','mirai-httpflooding','mirai-udpflooding']

In [294]:
X=pd.DataFrame([])
for i in flooding_list:
    X = X.append(X_reduced[i], ignore_index=True)

In [295]:
y=[]
for i in flooding_list:
    y.extend(label_dict[i])
y = np.array(y)

In [296]:
print(X.shape, len(y))

(1890579, 5) 1890579


In [273]:
# X=pd.DataFrame([])
# for i in dict_names:
#     X = X.append(X_dict[i])

In [274]:
# y=[]
# for i in dict_names:
#     y.extend(label_dict[i])
# y = np.array(y)

In [275]:
# print(len(X), len(y))

# 중간에 all.csv가져오는 코드 넣을것

In [276]:
# df = pd.read_csv("./datasets/iot-network-intrusion-dataset_190919/all.csv")

In [277]:
# X["Source"]=X["37"]+X["38"]
# X["Destination"]=X["37"]+X["38"]
# # X["No."] = df["No."]
# # X["Time"] = df["Time"]
# # X["Protocol"] = df["Protocol"]
# # X["Length"] = df["Length"]

In [278]:
# X_reduced = X[["Source", "Destination"]]

In [279]:
# X_reduced["Source"] = X_reduced["Source"].apply(to_int)
# X_reduced["Destination"] = X_reduced["Destination"].apply(to_int)

In [280]:
# X_reduced

In [281]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# df["Protocol"] = encoder.fit_transform(df["Protocol"])
# df["Source"] = encoder.fit_transform(df["Source"])
# df["Destination"] = encoder.fit_transform(df["Destination"])
# df["Info"] = encoder.fit_transform(df["Info"])

In [298]:
X["Destination"].value_counts()

210.89.164.90      1035380
192.168.0.16        282648
192.168.0.24        198069
192.168.0.13        124882
104.118.134.215      89478
                    ...   
222.195.98.45            1
111.55.216.93            1
111.192.32.31            1
222.98.242.120           1
222.217.15.225           1
Name: Destination, Length: 59527, dtype: int64

In [299]:
print(int(ipaddress.ip_address('192.168.0.0')))
print(192*(256**3) + 168*(256**2))

3232235520
3232235520


In [300]:
int(ipaddress.ip_address("192.168.0.1"))

3232235521

In [ ]:
for i in range(len(X["Source"])):
    try:
        X["Source"][i] = int(ipaddress.ip_address(X["Source"][i]))
    except ValueError:
        print(X["Source"][i])
try:
    X["Destination"] = int(ipaddress.ip_address(X["Destination"]))   
except ValueError:
    pass

C:\Users\hjhhi\anaconda3\envs\python3.6.8\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


EFMNetwo_d7:1c:56
Sichuani_4b:ae:ba
EFMNetwo_f2:ce:a7
SamsungE_d6:5d:9c
SamsungE_d6:5d:9c
SamsungE_d6:5d:9c
EFMNetwo_f2:ce:a7
EFMNetwo_f2:ce:a7
Sichuani_4b:ae:ba
EFMNetwo_d7:1c:56
EFMNetwo_f2:ce:a7
Sichuani_4b:ae:ba
SamsungE_d6:5d:9c
SamsungE_d6:5d:9c
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_cb:8d:af
Sichuani_4b:ae:ba
Sichuani_0b:ae:ba
Sichuani_4b:ae:ba
EFMNetwo_f2:ce:a7
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
EFMNetwo_32:a2:a7
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
EFMNetwo_f2:ce:a7
Sichuani_4b:ae:1a
bc:1c:01:b2:33:ba
EFMNetwo_f2:ce:a7
EFMNetwo_f2:ce:a7
EFMNetwo_f2:ce:a7
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
EFMNetwo_f2:ce:a7
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
96:92:05:c1:af:ba
EFMNetwo_92:cc:a7
Sichuani_4b:ae:ba
Sichuani_4b:ae:ba
bc:1c:01:4b:a6:ba
SamsungE_d6:5d:9c
EFMNetwo_f2:ce:a7
EFMNetwo_d7:1c:56
Sichuani_4b:ae:ba
Apple_c9:bf:fc
EFMNetwo_d7:1c:56
Apple_c9:bf:f

In [304]:
X

,SourcePort,DestinationPort,Source,Destination,Time
0,2048,43774,192.168.0.13,192.168.0.1,0.000000
1,0,45822,192.168.0.1,192.168.0.13,0.000545
2,9020,44144,192.168.0.13,192.168.0.23,0.008112
3,9020,44144,192.168.0.13,192.168.0.23,0.008999
4,44144,9020,192.168.0.23,192.168.0.13,0.011836
...,...,...,...,...,...
1890574,43238,443,192.168.0.24,104.118.134.215,465.053197
1890575,443,43238,104.118.134.215,192.168.0.24,465.099985
1890576,443,43238,104.118.134.215,192.168.0.24,465.099985
1890577,43238,443,192.168.0.24,104.118.134.215,465.101700


In [211]:
sum(X_reduced["mirai-ackflooding"]["SourcePort"]==0)

180

In [212]:
print(len(y), len(y[y==0]), len(y[y==1]))

1890579 790553 1100026


In [213]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [224]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.25)

In [225]:
X_train

,SourcePort,DestinationPort,Source,Destination,Time
874712,56361,10101,192.168.0.13,192.168.0.16,203.440783
47430,51993,8899,192.168.0.24,210.89.164.90,23.107873
119495,51998,8899,192.168.0.13,210.89.164.90,36.529309
853320,43238,443,192.168.0.24,104.118.134.215,184.050537
169141,65005,7777,192.168.0.24,210.89.164.90,249.552091
...,...,...,...,...,...
642369,60075,8899,192.168.0.13,210.89.164.90,71.277753
1020508,60065,8011,192.168.0.24,210.89.164.90,316.662195
189912,64782,9988,192.168.0.24,210.89.164.90,40.099535
258100,443,43238,104.118.134.215,192.168.0.24,354.675308


In [218]:
len(y_train)

1417934

In [220]:
###max_iter
model_logistic = LogisticRegression()
model_logistic.fit(X_train, y_train)

ValueError: could not convert string to float: '13.250.217.66'

In [32]:
model_knn = KNeighborsClassifier(n_neighbors=10)
model_knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [30]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

RandomForestClassifier()

In [33]:
model_knn.score(X_valid, y_valid)

0.9652698797989012

In [31]:
model_rf.score(X_valid, y_valid)

0.9671734322484022

In [45]:
X_train, X_valid, y_train, y_valid = train_test_split(X, np.array(y), test_size=0.25)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(10, activation="relu", input_shape=[X_train.shape[1]]),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data = (X_valid, y_valid))

Train on 2239495 samples, validate on 746499 samples
Epoch 1/10


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu", input_shape=[X_train.shape[1]]),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data = (X_valid, y_valid))

Train on 949691 samples, validate on 237423 samples
Epoch 1/20
949691/949691 [==============================] - 75s 79us/sample - loss: 0.1487 - accuracy: 0.9750 - val_loss: 0.0874 - val_accuracy: 0.9785
Epoch 2/20
949691/949691 [==============================] - 93s 98us/sample - loss: 0.0869 - accuracy: 0.9784 - val_loss: 0.0850 - val_accuracy: 0.9785
Epoch 3/20
949691/949691 [==============================] - 93s 98us/sample - loss: 0.0858 - accuracy: 0.9785 - val_loss: 0.0864 - val_accuracy: 0.9785
Epoch 4/20
949691/949691 [==============================] - 93s 98us/sample - loss: 0.0853 - accuracy: 0.9785 - val_loss: 0.0874 - val_accuracy: 0.9785
Epoch 5/20
949691/949691 [==============================] - 80s 84us/sample - loss: 0.0851 - accuracy: 0.9785 - val_loss: 0.0842 - val_accuracy: 0.9785
Epoch 6/20
949691/949691 [==============================] - 78s 82us/sample - loss: 0.0849 - accuracy: 0.9785 - val_loss: 0.0848 - val_accuracy: 0.9785
Epoch 7/20
949691/949691 [==========

In [34]:
flood_pred = model.predict_classes(X_valid)
result=["benign", "attacked"]
print(classification_report(y_valid,flood_pred , target_names=result))

NameError: name 'model' is not defined

In [102]:
flood_pred = model.predict_classes(X_valid)
result=["benign", "attacked"]
print(classification_report(y_valid,flood_pred , target_names=result))

              precision    recall  f1-score   support

      benign       1.00      1.00      1.00     35814
    attacked       1.00      1.00      1.00       186

    accuracy                           1.00     36000
   macro avg       1.00      1.00      1.00     36000
weighted avg       1.00      1.00      1.00     36000



In [30]:
model.trainable_variables

[<tf.Variable 'dense/kernel:0' shape=(66, 100) dtype=float32, numpy=
 array([[-0.21548988, -0.06131569, -0.05593153, ...,  0.009556  ,
          1.7233193 , -0.2308436 ],
        [-0.18978657, -0.25972167,  0.10120087, ..., -0.0030753 ,
         -2.4265988 , -0.25424433],
        [-0.02546336, -0.03507623, -0.1180388 , ...,  0.00665484,
         -1.0056187 , -0.19892313],
        ...,
        [-0.08734512, -0.02374357, -0.04575067, ..., -0.10786477,
          1.9088019 ,  0.13243084],
        [-0.08642702,  0.0873314 , -0.0180158 , ...,  0.05652766,
          0.4187118 ,  0.0130941 ],
        [-0.27137038,  0.19135252, -0.09057982, ...,  0.07734038,
          0.3805156 ,  0.05727315]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(100,) dtype=float32, numpy=
 array([-0.05285334, -0.12155698, -0.01935806, -0.6765026 , -0.07473051,
        -0.0804761 , -0.05523553, -0.02441452, -0.08331953, -0.05623414,
        -0.05350327, -0.17753392, -0.11969976, -0.0678602 , -0.05730273,
     